In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-txt/formatted_training_data.txt


# Introduction

# Installing necessary libraries

In [2]:
!pip install telethon
!pip install python-decouple
!pip install asyncio
!pip install config
!pip install transformers
!pip install datasets
!pip install tokenizers
!pip install peft
!pip install trl
!pip install tqdm

!pip install -q accelerate
!pip install -q -i https://pypi.org/simple/ bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 7.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.36.0-py3-none-any.whl size=680879 sha256=ed4bed01803862e091cbb892357456c026be2f1b3e79a0282e89a3dd7301be7b
  Stored in directory: /root/.cache/pip/wheels/4f/6f/86/da2c68242ca7dd92786186731ee763b7ecac8866666ac47170
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=2c8970c7eafb5a0c4ccbcf921b9f179191a4002695dd073b7106cef805119ba3
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 1.8 MB/s eta 0:00:00 0:00:01


In [30]:
from telethon.sync import TelegramClient
from telethon import TelegramClient
import pandas as pd
import json 
from pathlib import Path
import json
import os
import asyncio

from decouple import config
from loguru import logger
from telethon import TelegramClient
from telethon.tl.custom.dialog import Dialog
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.patched import Message

#For Fine Tuning
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer
from transformers import pipeline

ModuleNotFoundError: No module named 'telethon'

# Getting all messages of my channel on Telegram

In [ ]:
#You have to get the TELEGRAM_APP_ID and TELEGRAM_APP_HASH from telegram org
PHONE_NUMBER = 'your number with country code'
TELEGRAM_APP_ID = ""
TELEGRAM_APP_HASH = ""

# This is the name of the session file that will be created and stored in the working directory. Session files are used to store the state of the client, so that it can be resumed later so you dont have to login everytime you run the code
session_name = "session_1"

In [25]:
async def main(chat_name, limit):
    # "async with" creates asynchronous context managers
    # It is an extension of the “with” expression for use only in coroutines within asyncio programs
    async with TelegramClient(session_name, TELEGRAM_APP_ID, TELEGRAM_APP_HASH) as client:
        
        # Get chat info 
        chat_info = await client.get_entity(chat_name)
        
        # Get all the messages, given the limit
        # It will return the latest 5 messages if limit is 5
        messages = await client.get_messages(entity=chat_info, limit=limit)
        
        # return the results in a dictionary
        return ({"messages": messages, "channel": chat_info})

In [26]:
# limit=None will collect all the messages from nytimes Telegram channel (https://t.me/nytimes)
# This open an input box and ask you to input your phone number 
#  
chat_input = "Muslimbek_01"
results = await main(chat_name = chat_input, limit=None)

OperationalError: database is locked

In [26]:
results.keys()
messages = results['messages']
channel = results['channel']

print(len(messages))

731


As you can see I have posted 731 post on my telegram channel.

In [31]:
results["messages"][0].text

'**На этом у меня все! Спасибо, что были со мной на этой неделе! Мне было очень приятно делиться своими приключениями и полезной информацией с вами. Я всегда готов помочь вам с любыми вопросами об обучении, и я настоятельно рекомендую вам участвовать в семестровом обмене! Это открывает перед вами множество возможностей.**'

In [32]:
msg_list = [msg.to_dict() for msg in results["messages"]]

In [37]:
# Save results 
#Path(os.path.join("json_data")).mkdir(parents=True, exist_ok=True)
output_dir = '/kaggle/working/'
os.makedirs(output_dir, exist_ok=True)

out_path = os.path.join(output_dir, f"{chat_input}.json")

with open(out_path, "w") as f:
    json.dump(msg_list, f, default=str, ensure_ascii=False)

# Optionally, print the saved file path
print(f"JSON file saved to: {out_path}")

JSON file saved to: /kaggle/working/Muslimbek_01.json


In [38]:
df = pd.read_json('/kaggle/working/Muslimbek_01.json')
df.head()

,_,id,peer_id,date,message,out,mentioned,media_unread,silent,post,...,edit_date,post_author,grouped_id,reactions,restriction_reason,ttl_period,quick_reply_shortcut_id,effect,factcheck,action
0,Message,761,"{'_': 'PeerChannel', 'channel_id': 1466204272}",2024-05-27 11:05:16+00:00,"На этом у меня все! Спасибо, что были со мной ...",False,False,False,False,True,...,2024-05-27 11:26:11+00:00,NaN,NaN,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],NaN,NaN,NaN,NaN,NaN
1,Message,760,"{'_': 'PeerChannel', 'channel_id': 1466204272}",2024-05-27 11:05:16+00:00,А в последнем посте я хочу поделиться с Вами п...,False,False,False,False,True,...,2024-05-27 11:26:20+00:00,NaN,NaN,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],NaN,NaN,NaN,NaN,NaN
2,Message,759,"{'_': 'PeerChannel', 'channel_id': 1466204272}",2024-05-27 11:05:16+00:00,В предпоследним посте я хочу поделиться некото...,False,False,False,False,True,...,2024-05-27 11:26:26+00:00,NaN,NaN,"{'_': 'MessageReactions', 'results': [{'_': 'R...",[],NaN,NaN,NaN,NaN,NaN
3,Message,758,"{'_': 'PeerChannel', 'channel_id': 1466204272}",2024-05-26 13:44:15+00:00,Еще одно интересное мероприятие проходило в Ма...,False,False,False,False,True,...,2024-05-26 13:44:18+00:00,NaN,1.373385e+16,None,[],NaN,NaN,NaN,NaN,NaN
4,Message,757,"{'_': 'PeerChannel', 'channel_id': 1466204272}",2024-05-26 13:44:15+00:00,,False,False,False,False,True,...,2024-05-26 13:44:18+00:00,NaN,1.373385e+16,None,[],NaN,NaN,NaN,NaN,NaN


In [39]:
df.shape

(731, 40)

# Explaratory analysis of data

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

STOP = stopwords.words('english')
text = df["message"].dropna()
text = text[text.str.len()>0]
text

0      На этом у меня все! Спасибо, что были со мной ...
1      А в последнем посте я хочу поделиться с Вами п...
2      В предпоследним посте я хочу поделиться некото...
3      Еще одно интересное мероприятие проходило в Ма...
10     Еще одно интересное мероприятие было Easter Ev...
                             ...                        
723    Bir necha vaqt avval 6G internetini yaratish y...
725    Dasturchi (programmist) kim? \n\nDasturchi- ma...
726    Birinchi post bilan birga savol: IT nima degan...
727    Ulkan maqsadlar va ezgu niyatlar sababli ham u...
728    Assalomu aleykum. Kanalimizga xush kelibsiz! \...
Name: message, Length: 588, dtype: object

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
dtm=vectorizer.fit_transform(text)
vocab = vectorizer.vocabulary_

# Top 30 words 
sum_words = dtm.sum(axis=0) # a 1x9211 matrix
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

words_freq[0:30]

[('https', 24.48308473198657),
 ('youtu', 18.47896888162343),
 ('be', 18.356252000339808),
 ('va', 18.339575140820326),
 ('youtube', 17.36641923673008),
 ('uchun', 16.22914859988505),
 ('bu', 13.78930160151143),
 ('ushbu', 12.386216344500651),
 ('com', 12.269714405922143),
 ('bo', 12.0140930573792),
 ('shorts', 11.030827381099938),
 ('ham', 10.477123616210395),
 ('stop', 10.09720928250552),
 ('javob', 9.924699282798635),
 ('ma', 8.98412767955407),
 ('muslimbek_01', 8.85436353898065),
 ('daqiqa', 8.793609938632827),
 ('video', 8.281425623020072),
 ('bilan', 8.082916177679044),
 ('haqida', 8.044206723394842),
 ('yangi', 7.859540735950448),
 ('feature', 7.338270321320264),
 ('mumkin', 7.3368685834310465),
 ('share', 7.29860783679448),
 ('sima_company', 7.1446702952484165),
 ('bir', 6.965210967175569),
 ('eng', 6.796638681830508),
 ('siz', 6.332471658262747),
 ('kanalimiz', 6.30668920651635),
 ('obuna', 6.258674117868144)]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

plt.bar(words_counts[0:5])

In [59]:
# What are the different topics discussed?
from sklearn.cluster import KMeans

K = 10  # Number of clusters
kmeans = KMeans(n_clusters=K, random_state=42)
kmeans.fit(dtm)


order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(K):
    print(f"Cluster {i}:"),
    for ind in order_centroids[i, :10]:  # Print top 10 terms per cluster
        print(f" {terms[ind]}"),
    print()


Cluster 0:
 stop
 kasbga
 kasbni
 kashf
 kaskadli
 kastyumlar
 katalogini
 kategoriya
 katta
 kattakon

Cluster 1:
 shorts
 com
 youtube
 https
 feature
 share
 live
 iiwfydzzb0a
 jipeq_0ywkm
 h9jfmquu5ss

Cluster 2:
 daqiqa
 yoʻllash
 javob
 uchun
 oʻylash
 15
 keep
 keladigan
 kaskadli
 kastyumlar

Cluster 3:
 doimo
 kanalimiz
 bilan
 salomat
 sogʻ
 tabriklayman
 va
 bayram
 kuni
 muslimbek_01

Cluster 4:
 berib
 video
 haqida
 oʻtdim
 ma
 lumot
 videoda
 yangi
 youtube
 kanalimizda

Cluster 5:
 ham
 sababli
 va
 shu
 chatgpt
 ekan
 faqatgina
 bir
 muslimbek_01
 rossiya

Cluster 6:
 bo
 bu
 va
 uchun
 ushbu
 ham
 mumkin
 ko
 muslimbek_01
 javob

Cluster 7:
 va
 dasturlash
 html
 css
 sima_company
 web
 esa
 bu
 uchun
 obuna

Cluster 8:
 youtu
 be
 https
 lsm9gzknnam
 s0qcbry7czs
 f44nkbtliq8
 iuzyrhzn92a
 txdwsffde
 wb1_3lzaq1q
 knkvet6iqq4

Cluster 9:
 youtube
 video
 unutmang
 navbatdagi
 va
 ulashishni
 davom
 doʻstlarga
 ushbu
 kanalimizda



# Getting all messages that I sent on Telegram 

In [8]:
def get_dialogs(limit: int | None = 100) -> list[Dialog]:
    """Get all dialogs from the Telegram."""
    dialogs: list[Dialog] = client.get_dialogs(limit=limit)
    dialogs = [dialog for dialog in dialogs if dialog.is_user]  # remove groups or channels
    logger.info(f"Found {len(dialogs)} dialogs")
    return dialogs

In [9]:
def parse_messages(dialog: Dialog, limit: int = 1000) -> list[dict]:
    """Get all messages from the dialog."""
    all_messages_list = []
    offset_id = 0

    while True:
        messages: list[Message] = client(
            GetHistoryRequest(
                peer=dialog,
                offset_id=offset_id,
                offset_date=None,
                add_offset=0,
                limit=limit,
                max_id=0,
                min_id=0,
                hash=0,
            )
        ).messages
        if not messages:
            break

        all_messages_list.extend(
            {
                "date": message.date.isoformat(),
                "message": message.message,
                "out": message.out,
            }
            for message in messages
            # Filter audio or video content
            if message.message and not message.is_bot
        )
        offset_id = offset_id = messages[-1].id
    return all_messages_list

### Overall code

In [36]:
def save_to_json(file_to_save: list[dict[str, str]], file_name: str) -> None:
    with open(file_name, "w", encoding="utf-8") as f:
        json.dump(file_to_save, f, ensure_ascii=False)


async def get_dialogs(client, limit: int | None = 100) -> list[Dialog]:
    """Get all dialogs from Telegram."""
    dialogs = await client.get_dialogs(limit=limit)
    dialogs = [dialog for dialog in dialogs if dialog.is_user]  # Remove groups or channels
    logger.info(f"Found {len(dialogs)} dialogs")
    return dialogs


async def parse_messages(client, dialog: Dialog, limit: int = 1000) -> list[dict]:
    """Get all messages from the dialog."""
    all_messages_list = []
    offset_id = 0

    while True:
        history = await client(GetHistoryRequest(
            peer=dialog.entity,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0,
        ))
        messages: list[Message] = history.messages
        if not messages:
            break

        all_messages_list.extend(
            {
                "date": message.date.isoformat(),
                "message": message.message,
                "out": message.out,
            }
            for message in messages
            if message.message and not message.is_bot
        )
        offset_id = messages[-1].id
    return all_messages_list


async def main(limit: int = 1000):
    """Parse and save all private chat messages."""
    async with TelegramClient(SESSION_NAME, TELEGRAM_APP_ID, TELEGRAM_APP_HASH) as client:
        # Start the client
        await client.start()
        
        # Get all dialogs
        dialogs = await get_dialogs(client, limit=None)
        
        # Create directory if it does not exist
        os.makedirs("data", exist_ok=True)
        
        # Iterate through each dialog and save messages
        for dialog in dialogs:
            all_messages_list = await parse_messages(client, dialog, limit=limit)
            save_to_json(all_messages_list, f"data/{dialog.id}.json")
            logger.success(f"Saved {len(all_messages_list)} messages for {dialog.name or dialog.entity.username}")
        
        # Disconnect the client
        await client.disconnect()


if __name__ == "__main__":
    import asyncio

    # Set limit for the number of messages to collect per dialog
    limit = 1000 
    
    try:
        asyncio.run(main(limit))
    except RuntimeError as e:
        if "asyncio.run() cannot be called from a running event loop" in str(e):
            loop = asyncio.get_running_loop()
            loop.create_task(main(limit))
        else:
            raise


/tmp/ipykernel_33/1626331748.py:91: RuntimeWarning: coroutine 'main' was never awaited
  loop.create_task(main(limit))


In [17]:
df_json = pd.read_json('/kaggle/input/training-txt/combined_data.json')
pd.options.display.max_colwidth = None  # disable truncation

df_json.head(1)

,0,1,2,3,4,5,6,7,8,9,...,3724,3725,3726,3727,3728,3729,3730,3731,3732,3733
0,"{'date': '2022-05-28T03:43:36+00:00', 'message': 'Ok', 'out': False}","{'date': '2022-05-28T03:22:55+00:00', 'message': 'Bugun 22da oʻyin bor muhim', 'out': True}","{'date': '2022-05-24T05:32:40+00:00', 'message': 'Ok', 'out': False}","{'date': '2022-05-24T04:02:38+00:00', 'message': 'Aktiv boʻlsangiz boʻldi', 'out': True}","{'date': '2022-05-24T04:02:32+00:00', 'message': 'Óyin bóladigan kuni elon qilib qóyaman', 'out': True}","{'date': '2022-05-24T04:02:20+00:00', 'message': 'Online turnirlarda qatnashamiz', 'out': True}","{'date': '2022-05-24T03:29:52+00:00', 'message': 'Qanday bòladi zakovat jamodagilar bn tanishtiring', 'out': False}","{'date': '2022-05-24T03:27:29+00:00', 'message': 'Men ham ingliz tili òrganib chet elda magistraturani òqish niyatidaman', 'out': False}","{'date': '2022-05-24T03:26:43+00:00', 'message': 'Zòrqu', 'out': False}","{'date': '2022-05-24T03:08:21+00:00', 'message': 'Ha Men Norvegiyada exchange student sifatida o`qiyman', 'out': True}",...,None,None,None,None,None,None,None,None,None,None


# Fine tuning model

**I wanted to use small models as I had small number of time**

In [3]:
tinyllama = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

tokenizer = AutoTokenizer.from_pretrained(tinyllama)
model = AutoModelForCausalLM.from_pretrained(tinyllama)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**But in the end I decided to use Gpt2 as I have 3 languages Russian, English, Uzbek that I sent messages and this model is universal for these languages.**

In [29]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

def fine_tune_gpt2():
    # Load pre-trained model and tokenizer
    model_name = "gpt2"
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # Load dataset
    def load_dataset(file_path, tokenizer, block_size=128):
        cache_dir = '/kaggle/working/cache'
        os.makedirs(cache_dir, exist_ok=True)
        
        dataset = TextDataset(
            tokenizer=tokenizer,
            file_path=file_path,
            block_size=block_size,
            cache_dir="/kaggle/working'"
        )
        return dataset

    file_path = "/kaggle/input/training-txt/formatted_training_data.txt"
    dataset = load_dataset(file_path, tokenizer)

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir="/kaggle/working/results",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=1, 
        gradient_accumulation_steps=4, 
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='/kaggle/working/logs',
        dataloader_num_workers=4, 
        gradient_checkpointing=True, 
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()

    # Save the model
    trainer.save_model("/kaggle/working/fine_tuned_model")
    tokenizer.save_pretrained("/kaggle/working/fine_tuned_model")

fine_tune_gpt2()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,3.039500
1000,2.580500
1500,2.373900
2000,2.258600


In [34]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_response(prompt, model, tokenizer, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_model")

# Example usage
prompt = "Muslimbek"
response = generate_response(prompt, model, tokenizer)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Muslimbek 
 Ha
Ha ertaga ikki izga qilishni
Yaxshi icha   yaxshimisiz? 


📌 Turnir qilib qoʻyichib qolib ketdi
Turnir ichi  kunlari  kelganimiz
Yo‘q  yo’q,  qiziqarli  boʼladi.  Yo


# Uploading to HugginFace

In [35]:
!pip install huggingface_hub

In [41]:
from huggingface_hub import login
login()

In [44]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "MrSimple07/my-gpt2-model-for-clone"

api.create_repo(repo_id, private=False)  

api.upload_folder(
    folder_path="./fine_tuned_model",
    repo_id=repo_id,
    repo_type="model",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MrSimple07/my-gpt2-model-for-clone/commit/9c61c5fe760269d567bc5a7f3bd9ef90ac6d59c9', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9c61c5fe760269d567bc5a7f3bd9ef90ac6d59c9', pr_url=None, pr_revision=None, pr_num=None)

# Streamlit

In [57]:
!pip install streamlit pyngrok transformers torch

In [58]:
import streamlit as st
from pyngrok import ngrok
import nest_asyncio
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Apply nest_asyncio to allow asyncio to work in Jupyter
nest_asyncio.apply()

In [60]:
@st.cache_resource
def load_model():
    model = GPT2LMHeadModel.from_pretrained("MrSimple07/my-gpt2-model-for-clone")
    tokenizer = GPT2Tokenizer.from_pretrained("MrSimple07/my-gpt2-model-for-clone")
    return model, tokenizer

model, tokenizer = load_model()

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [61]:
def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def streamlit_app():
    st.title("Your Personalized Chatbot")

    user_input = st.text_input("You:", "")
    if st.button("Send"):
        if user_input:
            response = generate_response(user_input)
            st.text_area("Bot:", value=response, height=100, max_chars=None, key=None)
        else:
            st.warning("Please enter a message.")

# Write the app content to a file
with open("streamlit_app.py", "w") as f:
    f.write("""
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer

@st.cache_resource
def load_model():
    model = GPT2LMHeadModel.from_pretrained("MrSimple07/my-gpt2-model")
    tokenizer = GPT2Tokenizer.from_pretrained("MrSimple07/my-gpt2-model")
    return model, tokenizer

model, tokenizer = load_model()

def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    return tokenizer.decode(output[0], skip_special_tokens=True)

st.title("Your Personalized Chatbot")

user_input = st.text_input("You:", "")
if st.button("Send"):
    if user_input:
        response = generate_response(user_input)
        st.text_area("Bot:", value=response, height=100, max_chars=None, key=None)
    else:
        st.warning("Please enter a message.")
    """)

In [ ]:
get_ipython().system('streamlit run streamlit_app.py')

In [62]:
!streamlit run streamlit_app.py &>/dev/null&
public_url = ngrok.connect(port='8501')
print(f' * Running on {public_url}')

import time
while True:
    time.sleep(60)
    print("Keeping notebook running...")

OSError: Background processes not supported.

In [50]:
!streamlit run "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py"